
# Unit 2 — Team Classification (Flights, BQML)

**Goal (team):** Build an *ops-ready* classifier in **BigQuery ML** to predict **`diverted`** on U.S. flights. Minimal handholding by design.

**What you deliver (inside this notebook):**
- One **LOGISTIC_REG** model (baseline), one **engineered** model using `TRANSFORM`
- **Evaluation** via `ML.EVALUATE` and **confusion matrices** (default 0.5 + your custom threshold)
- **Threshold choice** + 3–5 sentence ops justification
- Embedded **rubric** below (self-check before submission)

> Choose *one* dataset table that exists at your institution:  
> • `bigquery-public-data.faa.us_flights` **or** `bigquery-public-data.flights.*`  
> Make sure the table has `carrier`, `dep_delay`, `arr_delay` (for filters), `origin`, `dest`, `diverted` (or equivalent).


In [ ]:

# --- Minimal setup (edit 3 vars) ---
from google.colab import auth
auth.authenticate_user()

import os
from google.cloud import bigquery

PROJECT_ID = "YOUR_PROJECT_ID"      # e.g., mgmt-467-47888
REGION     = "us-central1"
TABLE_PATH = "bigquery-public-data.faa.us_flights"   # or your `bigquery-public-data.flights` table/view

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"]     = REGION
bq = bigquery.Client(project=PROJECT_ID)

print("BQ Project:", PROJECT_ID)
print("Source table:", TABLE_PATH)


### Quick sanity check

In [ ]:

preview_sql = f"SELECT * FROM `{TABLE_PATH}` LIMIT 5"
bq.query(preview_sql).result().to_dataframe()



## 1) Canonical mapping (adjust as needed)
Map to a minimal schema used in the rest of the notebook:
- `flight_date` (DATE), `dep_delay` (NUM), `distance` (NUM), `carrier` (STRING), `origin` (STRING), `dest` (STRING), `diverted` (BOOL)


In [ ]:

# Adjust ONLY if your table uses different column names.
CANONICAL_BASE_SQL = f'''
WITH canonical_flights AS (
  SELECT
    CAST(COALESCE(flight_date, date) AS DATE) AS flight_date,
    CAST(dep_delay AS FLOAT64) AS dep_delay,
    CAST(distance  AS FLOAT64) AS distance,
    CAST(carrier   AS STRING)  AS carrier,
    CAST(origin    AS STRING)  AS origin,
    CAST(COALESCE(dest, destination) AS STRING) AS dest,
    CAST((CASE WHEN SAFE_CAST(diverted AS INT64)=1 OR LOWER(CAST(diverted AS STRING))='true' THEN TRUE ELSE FALSE END) AS BOOL) AS diverted
  FROM `{TABLE_PATH}`
  WHERE dep_delay IS NOT NULL
)
'''
print(CANONICAL_BASE_SQL[:600] + "\n...")


### 2) Split (80/20)

In [ ]:

SPLIT_CLAUSE = r'''
, split AS (
  SELECT cf.*,
         CASE WHEN RAND(12345) < 0.8 THEN 'TRAIN' ELSE 'EVAL' END AS split
  FROM canonical_flights cf
)
'''
print(SPLIT_CLAUSE)



## 3) Baseline model — LOGISTIC_REG (`diverted`)
Use **only** a small set of signals for the baseline (keep it honest).


In [ ]:

MODEL_BASE = f"{PROJECT_ID}.unit2_flights.clf_diverted_base"

sql_baseline = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.unit2_flights`;

CREATE OR REPLACE MODEL `{MODEL_BASE}`
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['diverted']) AS
SELECT
  diverted,
  dep_delay, distance, carrier, origin, dest,
  EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
FROM split
WHERE split='TRAIN'
;

SELECT * FROM ML.EVALUATE(
  MODEL `{MODEL_BASE}`,
  (SELECT
     diverted,
     dep_delay, distance, carrier, origin, dest,
     EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
   FROM split WHERE split='EVAL')
);
'''
job = bq.query(sql_baseline); _ = job.result()
print("Baseline model trained:", MODEL_BASE)


### Confusion matrix — default 0.5 threshold

In [ ]:

cm_default_sql = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

WITH scored AS (
  SELECT
    cf.diverted AS label,
    p.predicted_diverted AS pred_label,
    p.predicted_diverted_probs[OFFSET(0)].prob AS score
  FROM split cf
  JOIN ML.PREDICT(MODEL `{MODEL_BASE}`,
      (SELECT dep_delay, distance, carrier, origin, dest, EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
       FROM split)) AS p
  ON TRUE
  WHERE split='EVAL'
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_default_sql).result().to_dataframe()


### Confusion matrix — your custom threshold

In [ ]:

CUSTOM_THRESHOLD = 0.75  # TODO: justify in ops terms

cm_thresh_sql = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

WITH scored AS (
  SELECT
    cf.diverted AS label,
    CAST(score >= {CUSTOM_THRESHOLD} AS BOOL) AS pred_label
  FROM (
    SELECT cf.*, p.predicted_diverted_probs[OFFSET(0)].prob AS score
    FROM split cf
    JOIN ML.PREDICT(MODEL `{MODEL_BASE}`,
          (SELECT dep_delay, distance, carrier, origin, dest, EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
           FROM split)) AS p
    ON TRUE
    WHERE split='EVAL'
  )
)
SELECT
  SUM(CASE WHEN label=TRUE  AND pred_label=TRUE  THEN 1 ELSE 0 END) AS TP,
  SUM(CASE WHEN label=FALSE AND pred_label=TRUE  THEN 1 ELSE 0 END) AS FP,
  SUM(CASE WHEN label=TRUE  AND pred_label=FALSE THEN 1 ELSE 0 END) AS FN,
  SUM(CASE WHEN label=FALSE AND pred_label=FALSE THEN 1 ELSE 0 END) AS TN
FROM scored;
'''
bq.query(cm_thresh_sql).result().to_dataframe()



## 4) Engineered model — `TRANSFORM` (same label, stricter bar)
Create **route**, extract **day_of_week**, and **bucketize dep_delay**. Compare metrics to baseline.


In [ ]:

MODEL_XFORM = f"{PROJECT_ID}.unit2_flights.clf_diverted_xform"

sql_xform = f'''
{CANONICAL_BASE_SQL}
{SPLIT_CLAUSE}

CREATE OR REPLACE MODEL `{MODEL_XFORM}`
TRANSFORM (
  CONCAT(origin, '-', dest) AS route,
  EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week,
  CASE
    WHEN dep_delay < -5  THEN 'early'
    WHEN dep_delay <=  5 THEN 'on_time'
    WHEN dep_delay <= 15 THEN 'minor'
    WHEN dep_delay <= 45 THEN 'moderate'
    ELSE 'major'
  END AS dep_delay_bucket,
  dep_delay, distance, carrier, origin, dest
)
OPTIONS (MODEL_TYPE='LOGISTIC_REG', INPUT_LABEL_COLS=['diverted']) AS
SELECT * FROM split WHERE split='TRAIN'
;

SELECT 'baseline' AS model_version, * FROM ML.EVALUATE(
  MODEL `{MODEL_BASE}`,
  (SELECT
     diverted,
     dep_delay, distance, carrier, origin, dest,
     EXTRACT(DAYOFWEEK FROM flight_date) AS day_of_week
   FROM split WHERE split='EVAL')
)
UNION ALL
SELECT 'engineered' AS model_version, * FROM ML.EVALUATE(
  MODEL `{MODEL_XFORM}`,
  (SELECT * FROM split WHERE split='EVAL')
);
'''
job = bq.query(sql_xform); _ = job.result()
print("Engineered model trained:", MODEL_XFORM)



### Write-up (concise)
- **Threshold chosen & ops rationale:** …  
- **Baseline vs engineered — observed changes in AUC/precision/recall:** …  
- **Risk framing:** cost of FP vs FN for diversion planning; what is your acceptable FN-rate? …



---

## Rubric (Flights, 100 pts)
**Team-only deliverable in this notebook**

- Baseline LOGISTIC_REG + evaluation (AUC + confusion @0.5) — **20**  
- Custom threshold confusion matrix + ops justification — **20**  
- Engineered model with `TRANSFORM` (route, DOW, delay bucket) — **20**  
- Comparison table (baseline vs engineered) + 3–5 sentence interpretation — **20**  
- Reproducibility: parameters clear, no hidden magic; schema mapping documented — **10**  
- Governance notes: assumptions/limitations + slices you would monitor — **10**

> **Strictness:** No screenshots; use actual results cells. Keep explanations concise (bullet points OK).
